# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [36]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [37]:
input_data='./input_data/'
#output_data='./output_data/2020/'
#output_data='./output_data/'
output_data='./output_data/2022/'

In [38]:
model_name='Base Case'
dryrun=False #Dont write output files

In [39]:
#models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
#models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
models=pd.read_csv(input_data + "2022/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="/scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity'] #Base 180MW nominal capacity. Only used to calculate 'nominal capacity factor'
units=4 #Design no of units
units_avialable=model['UnitsAvailable'] #Available no of units - to see what happens during testing/commissioning

rated_flow_unit=model['RatedDischarge'] #250
max_flow_unit=rated_flow_unit*model['MaximumLoadFactor'] #275
canal_capacity=max_flow_unit*4 #canal capacity 1100 cumec 
min_flow_unit=model['MinimumFlowUnit'] #50cumec minimum flow for a turbine
fixed_tailwater_level=model['FixedTailwater']#False #970 
head_minimum=model['MinimumHead'] #minimum head 10m

rated_power_unit=rated_flow_unit*model['RatedHead']*model['RatedTurbineEfficiency']*model['RatedGeneratorEfficiency']*9.81/1000

model

OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
HeadpondLift                               0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
RatedPower                            49.182
Mean                                  838165
P95                                   719611
P90                                   757122
P75                                   796138
P50                                   843034
P25                                   878919
P10                                   929634
P5        

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [49]:
daily=pd.read_csv(output_data + out_prefix + '_pe_daily.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)

In [50]:
#daily=daily.drop(daily.loc['2020-05-01':].index)
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u,ExceedanceRnd
Date,,,,,,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0000,1.384095,138.409487,0.998,1924,10,1,1924.10,1924,...,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600,1.000
1924-10-02,1924-10-13,100.0000,1.384095,138.409487,0.998,1924,10,2,1924.10,1924,...,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600,1.000
1924-10-03,1924-10-14,100.0000,1.384095,138.409487,0.998,1924,10,3,1924.10,1924,...,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600,1.000
1924-10-04,1924-10-15,100.0000,1.384095,138.409487,0.998,1924,10,4,1924.10,1924,...,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600,1.000
1924-10-05,1924-10-16,100.0000,1.384095,138.409487,0.998,1924,10,5,1924.10,1924,...,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-26,2020-05-07,4444.4855,1.018123,4525.031230,0.026,2020,4,26,2020.04,2019,...,25.930442,25.930442,103.721769,2489.322462,0.022922,29.476339,57.826543,83.858150,104.667744,0.025
2020-04-27,2020-05-08,4427.0109,1.018123,4507.239944,0.026,2020,4,27,2020.04,2019,...,26.018262,26.018262,104.073047,2497.753120,0.023090,29.564512,58.002915,84.122587,105.019208,0.025
2020-04-28,2020-05-09,4392.1861,1.020200,4480.910258,0.027,2020,4,28,2020.04,2019,...,26.148699,26.148699,104.594796,2510.275095,0.023342,29.695421,58.264783,84.515249,105.541194,0.025


## Monthly


In [45]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
   FlowAvailableForGeneration_min=('FlowAvailableForGeneration','min')
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,Flow_difference,Flow_difference_pct,EWRChannelA,EWRChannelC,...,PotentialPower4u,ExceedanceRnd,Power_max,Power_min,Turbines_max,Turbines_min,Energy,FlowAvailableForGeneration_min,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
1924.10,136.771526,0.998452,1924.0,10.0,1924.0,1.0,0.610130,0.020579,28.193548,4.322581,...,224.359226,0.999839,18.828159,0.000000,1,0,8767.783860,41.409487,31,0.065470
1924.11,153.655969,0.992400,1924.0,11.0,1924.0,2.0,2.353660,0.014794,23.000000,4.000000,...,224.488134,0.992333,31.166339,17.167260,1,1,15906.415492,82.587158,30,0.122735
1924.12,276.157068,0.831323,1924.0,12.0,1924.0,3.0,9.160736,0.029641,26.000000,4.000000,...,222.365311,0.831774,81.571064,29.453711,2,1,35543.558652,134.122845,31,0.265409
1925.01,704.059970,0.459323,1925.0,1.0,1924.0,4.0,44.430930,0.055882,37.129032,5.677419,...,209.716186,0.459355,192.336429,73.315246,4,2,88999.965733,345.554562,31,0.664576
1925.02,1210.241015,0.279000,1925.0,2.0,1924.0,5.0,25.090361,0.020239,49.000000,8.000000,...,192.334639,0.279286,191.342471,177.468973,4,4,123652.229395,964.382002,28,1.022257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019.12,306.896401,0.779677,2019.0,12.0,2019.0,3.0,4.898025,0.015547,29.225806,4.645161,...,221.756764,0.779516,70.824087,37.757472,2,1,38412.501873,171.213858,31,0.286832
2020.01,518.101218,0.550290,2020.0,1.0,2019.0,4.0,8.397484,0.016302,32.387097,5.000000,...,215.956849,0.550000,114.139122,68.868252,3,2,68024.346100,322.805464,31,0.507948
2020.02,852.617607,0.389931,2020.0,2.0,2019.0,5.0,21.269790,0.022228,36.000000,6.000000,...,204.694804,0.390345,190.694690,114.653030,4,3,100935.568867,565.032470,29,0.805680


## Yearly

In [43]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),   
   PowerTurbine1_max=('PowerTurbine1', 'max'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [44]:
#yearly=yearly.drop(yearly.loc['2019':].index)
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,PowerTurbine1_max,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,,
2015,2983.091137,173.175160,0.500494,0.067020,25.390592,14.484177,60.585296,973.578921,964.401315,190.801349,21.077852,792846.001389,0.502820
2016,3912.450050,158.591340,0.485607,0.047048,25.389885,12.600537,59.007904,975.462561,964.401315,191.837298,19.265001,855274.247527,0.542411
2017,4855.789688,171.188037,0.555972,0.055607,25.390592,10.970641,60.585296,977.092457,964.401315,188.857727,20.659607,843033.897521,0.534649
2018,1103.622889,158.591340,0.523314,0.109287,25.437359,20.415804,60.407077,968.222504,964.401315,180.553039,18.010865,727306.550364,0.461255
2019,4709.551434,130.594892,0.398178,0.054678,25.375426,11.208360,60.407077,976.854738,964.401315,190.694690,16.152746,422387.152414,0.267876


In [12]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))

In [13]:
yearly['PropDays1uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterYear').count()
yearly['PropDays2uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterYear').count()
yearly['PropDays3uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterYear').count()
yearly['PropDays4uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterYear').count()

In [14]:
daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

WaterYear
1924.0     57
1925.0     66
1926.0     53
1927.0     67
1928.0    104
         ... 
2014.0    129
2015.0     61
2016.0     54
2017.0     40
2019.0     63
Name: Date, Length: 87, dtype: int64

In [15]:
yearly['FstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)


## Power and energy exceedance

In [16]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,194.044880
0.05,182.469492
0.10,173.986975


In [17]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.027124e+06
0.05,9.658875e+05
0.10,9.296344e+05


In [18]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [19]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [20]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(plant_capacity*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,859699.914800,0.545218
1926,784396.018845,0.497461
1927,858896.868857,0.544709
1928,835999.989452,0.530188
1929,763337.011596,0.484105


## Monthly Day
15th of every month

In [21]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [22]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-19,2020,4,19,2019,4539,0.24,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.4,2482
2020-04-20,2020,4,20,2019,4599,0.24,2,70,14,18,...,0.85,0.85,0.85,0.85,25.6,25.6,25.6,25.6,102.3,2454
2020-04-21,2020,4,21,2019,4657,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.3,25.3,25.3,25.3,101.1,2427
2020-04-22,2020,4,22,2019,4709,0.13,2,70,14,18,...,0.84,0.84,0.84,0.84,25.0,25.0,25.0,25.0,100.1,2403
2020-04-23,2020,4,23,2019,4657,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.3,25.3,25.3,25.3,101.1,2427
2020-04-24,2020,4,24,2019,4657,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.3,25.3,25.3,25.3,101.1,2427
2020-04-25,2020,4,25,2019,4539,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.4,2482
2020-04-26,2020,4,26,2019,4525,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,25.9,25.9,25.9,25.9,103.7,2489
2020-04-27,2020,4,27,2019,4507,0.13,2,70,14,18,...,0.85,0.85,0.85,0.85,26.0,26.0,26.0,26.0,104.1,2497


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [23]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(plant_capacity*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  \
WaterYear WaterWeek                                             
2019      20          802.675285    0.404429         9.762452   
          21          896.461613    0.369286        14.421316   
          22         1189.264150    0.287571        64.599124   
          23         2064.093106    0.173429       188.575019   
          24         3397.133952    0.068286       183.144410   
          25         4273.237093    0.032571        80.664268   
          26         3997.761524    0.039429        42.199587   
          27         3886.363521    0.043143        37.961285   
          28         4077.410565    0.037429        40.817595   
          29         4449.955083    0.027857        48.957590   
          30         4607.704148    0.024000        44.687618   
          31         4444.073146    0.028333        39.563034   

                     Flow_difference_pct  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                  
2019      20                    0.012065    36.000000     6.000000   
          21                    0.016118    36.000000     6.000000   
          22                    0.053234    39.714286     6.857143   
          23                    0.087571    49.000000     9.000000   
          24                    0.053502    57.571429    10.714286   
          25                    0.019010    59.000000    11.000000   
          26                    0.010534    59.000000    11.000000   
          27                    0.009766    68.428571    13.571429   
          28                    0.009822    70.000000    14.000000   
          29                    0.010961    70.000000    14.000000   
          30                    0.009702    70.000000    14.000000   
          31                    0.008951    70.000000    14.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  ...  \
WaterYear WaterWeek                                                      ...   
2019      20            5.000000     4.000000     54.000000  105.000000  ...   
          21            5.000000     4.000000     54.000000  105.000000  ...   
          22            6.142857     5.428571     69.142857  127.285714  ...   
          23            9.000000     9.000000    107.000000  183.000000  ...   
          24           11.571429    12.428571    138.714286  231.000000  ...   
          25           12.000000    13.000000    144.000000  239.000000  ...   
          26           12.000000    13.000000    144.000000  239.000000  ...   
          27           17.142857    23.285714    164.571429  287.000000  ...   
          28           18.000000    25.000000    168.000000  295.000000  ...   
          29           18.000000    25.000000    168.000000  295.000000  ...   
          30           18.000000    25.000000    168.000000  295.000000  ...   
          31           18.000000    25.000000    168.000000  295.000000  ...   

                     HeadTurbine1_max  HeadTurbine1_min  LevelTailwater_max  \
WaterYear WaterWeek                                                           
2019      20                22.261402         21.886660          967.167066   
          21                21.757517         21.285966          967.579372   
          22                21.364491         18.885022          969.249856   
          23                18.711068         15.106451          972.956647   
          24                14.859692         12.452596          975.610502   
          25                12.179992         11.778503          976.284595   
          26                12.643117         12.240611          975.822487   
          27                12.806989         12.473410          975.589688   
          28                12.452596         11.978517          976.084581   
          29                11.917977         11.391348          976.671750   
          30                11.545667         11.208360          97

In [24]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [25]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Power_P95']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.05)
calmonthly['Power_P90']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.10)

calmonthly['FlowAvailableForGeneration_P95']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.05)
calmonthly['FlowAvailableForGeneration_P50']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.5)

calmonthly['PotentialPower1u_P95']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower1u_P50']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower2u_P95']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower2u_P50']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower3u_P95']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower3u_P50']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower4u_P95']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower4u_P50']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.5)


calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower1u_P95,PotentialPower1u_P50,PotentialPower2u_P95,PotentialPower2u_P50,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.485351,0.393500,0.397798,0.272949,120.0,93.148185,97.000000,52.0,...,56.910001,57.311466,113.306496,114.143392,168.576066,169.916411,221.031322,223.020883,6818.411885,0.290475
2,Nov,11,0.400612,0.331485,0.334500,0.214463,114.0,89.079861,93.000000,50.0,...,56.534561,57.220033,112.526828,113.952538,167.334567,169.610139,219.204737,222.564891,9026.525116,0.309559
3,Dec,12,0.287921,0.241238,0.241329,0.153900,126.0,100.028226,104.000000,59.0,...,54.550172,56.569237,108.436192,112.599471,160.894405,167.451772,209.896818,219.380664,15035.781922,0.286000
4,Jan,1,0.222306,0.174756,0.178740,0.097430,150.0,117.400202,120.000000,73.0,...,49.950758,55.054382,99.065062,109.471368,146.410363,162.513862,189.584693,212.213716,20529.409717,0.241474
5,Feb,2,0.186837,0.127406,0.136113,0.023905,197.0,138.252617,137.000000,85.0,...,36.982453,52.343979,72.880604,103.922553,106.530440,153.872005,135.052847,199.940275,18422.513908,0.185630
6,Mar,3,0.179385,0.094252,0.090588,0.040404,315.0,193.098790,192.032258,107.0,...,24.184082,42.247860,47.275385,83.468013,56.413017,122.544898,29.404406,156.685057,30459.559441,0.293722
7,Apr,4,0.160675,0.084645,0.079915,0.050051,364.0,231.197917,217.000000,125.0,...,26.631236,38.176838,52.141986,75.275321,75.351671,110.135238,82.897668,139.879657,27053.986732,0.276830
8,May,5,0.218068,0.129659,0.126994,0.065455,340.0,263.163667,276.000000,147.0,...,33.485919,43.083220,65.866209,85.154445,95.961707,125.112792,120.877481,160.195063,16825.474391,0.147723
9,Jun,6,0.369599,0.196463,0.191664,0.108597,293.0,211.400000,210.000000,119.0,...,44.109459,50.848344,87.225898,100.874909,128.266121,149.160686,164.503262,193.331078,23038.241463,0.217480


In [26]:
calmonthly[['Power_P90','Power_P95']]

,Power_P90,Power_P95
WaterMonth,,
1,22.330549,20.274339
2,26.305734,23.399802
3,43.422287,39.759278
4,76.709515,69.581829
5,107.909476,96.855247
6,96.151995,0.000000
7,96.628359,87.102914
8,120.730098,107.562746
9,82.321016,64.807346


In [27]:
calmonthly['PropYearsMinAvailFlowGt1UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt2UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt3UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGtCanalRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>canal_capacity).groupby('WaterMonth').count()/len(yearly.index)
calmonthly.fillna(0)
calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.485351,0.393500,0.397798,0.272949,120.0,93.148185,97.000000,52.0,...,168.576066,169.916411,221.031322,223.020883,6818.411885,0.290475,0.031579,NaN,NaN,NaN
2,Nov,11,0.400612,0.331485,0.334500,0.214463,114.0,89.079861,93.000000,50.0,...,167.334567,169.610139,219.204737,222.564891,9026.525116,0.309559,0.052632,NaN,NaN,NaN
3,Dec,12,0.287921,0.241238,0.241329,0.153900,126.0,100.028226,104.000000,59.0,...,160.894405,167.451772,209.896818,219.380664,15035.781922,0.286000,0.273684,0.021053,NaN,NaN
4,Jan,1,0.222306,0.174756,0.178740,0.097430,150.0,117.400202,120.000000,73.0,...,146.410363,162.513862,189.584693,212.213716,20529.409717,0.241474,0.978947,0.231579,0.052632,NaN
5,Feb,2,0.186837,0.127406,0.136113,0.023905,197.0,138.252617,137.000000,85.0,...,106.530440,153.872005,135.052847,199.940275,18422.513908,0.185630,1.010526,0.863158,0.368421,0.126316
6,Mar,3,0.179385,0.094252,0.090588,0.040404,315.0,193.098790,192.032258,107.0,...,56.413017,122.544898,29.404406,156.685057,30459.559441,0.293722,1.010526,0.989474,0.810526,0.557895
7,Apr,4,0.160675,0.084645,0.079915,0.050051,364.0,231.197917,217.000000,125.0,...,75.351671,110.135238,82.897668,139.879657,27053.986732,0.276830,1.010526,1.010526,0.968421,0.821053
8,May,5,0.218068,0.129659,0.126994,0.065455,340.0,263.163667,276.000000,147.0,...,95.961707,125.112792,120.877481,160.195063,16825.474391,0.147723,1.000000,0.947368,0.894737,0.726316
9,Jun,6,0.369599,0.196463,0.191664,0.108597,293.0,211.400000,210.000000,119.0,...,128.266121,149.160686,164.503262,193.331078,23038.241463,0.217480,0.947368,0.589474,0.263158,0.063158


In [28]:
calmonthly['PropYears1uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears2uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears3uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears4uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterMonth').count()/len(yearly.index)

calmonthly.fillna(0)
calmonthly


,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow,PropYears1uPotentialGtRatedPower,PropYears2uPotentialGtRatedPower,PropYears3uPotentialGtRatedPower,PropYears4uPotentialGtRatedPower
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.485351,0.393500,0.397798,0.272949,120.0,93.148185,97.000000,52.0,...,6818.411885,0.290475,0.031579,NaN,NaN,NaN,1.010526,1.010526,1.010526,1.010526
2,Nov,11,0.400612,0.331485,0.334500,0.214463,114.0,89.079861,93.000000,50.0,...,9026.525116,0.309559,0.052632,NaN,NaN,NaN,1.010526,1.010526,1.010526,1.010526
3,Dec,12,0.287921,0.241238,0.241329,0.153900,126.0,100.028226,104.000000,59.0,...,15035.781922,0.286000,0.273684,0.021053,NaN,NaN,1.010526,1.010526,1.010526,1.010526
4,Jan,1,0.222306,0.174756,0.178740,0.097430,150.0,117.400202,120.000000,73.0,...,20529.409717,0.241474,0.978947,0.231579,0.052632,NaN,0.968421,0.957895,0.947368,0.915789
5,Feb,2,0.186837,0.127406,0.136113,0.023905,197.0,138.252617,137.000000,85.0,...,18422.513908,0.185630,1.010526,0.863158,0.368421,0.126316,0.747368,0.736842,0.705263,0.652632
6,Mar,3,0.179385,0.094252,0.090588,0.040404,315.0,193.098790,192.032258,107.0,...,30459.559441,0.293722,1.010526,0.989474,0.810526,0.557895,0.315789,0.305263,0.284211,0.231579
7,Apr,4,0.160675,0.084645,0.079915,0.050051,364.0,231.197917,217.000000,125.0,...,27053.986732,0.276830,1.010526,1.010526,0.968421,0.821053,0.157895,0.136842,0.094737,0.073684
8,May,5,0.218068,0.129659,0.126994,0.065455,340.0,263.163667,276.000000,147.0,...,16825.474391,0.147723,1.000000,0.947368,0.894737,0.726316,0.168421,0.168421,0.136842,0.126316
9,Jun,6,0.369599,0.196463,0.191664,0.108597,293.0,211.400000,210.000000,119.0,...,23038.241463,0.217480,0.947368,0.589474,0.263158,0.063158,0.673684,0.642105,0.557895,0.473684


## Flow duration curve

In [29]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8809.879593,0.001431,0.761174,141.407689,0.016459,0.026667,1.097222,66.083333,12.430556,14.861111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6389.293248,0.004989,0.552035,109.134863,0.016749,0.043611,1.100000,72.427778,13.838889,17.688889,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.506935,22.466286,18.795424,0.000000
0.010,5563.531659,0.009983,0.480689,65.252467,0.011744,0.083204,1.320442,72.668508,13.574586,17.591160,...,8.685289,8.685289,8.685289,34.741155,833.787711,0.006379,24.862055,48.608370,70.067072,44.425684
0.015,5203.203065,0.014944,0.449557,89.050316,0.017197,0.101611,1.388889,71.366667,12.933333,16.600000,...,22.836167,22.836167,22.836167,91.344667,2192.271998,0.017571,26.352158,51.582774,74.510725,92.278791
0.020,4839.769106,0.020054,0.418156,93.641639,0.019328,0.110598,1.402174,69.554348,13.195652,16.597826,...,24.421997,24.421997,24.421997,97.687988,2344.511712,0.020200,27.958747,54.792348,79.312530,98.635084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,181.295707,0.980401,0.015664,2.697233,0.014865,0.893882,4.447368,28.546053,4.355263,2.782895,...,NaN,NaN,NaN,24.774158,594.579785,0.136628,57.650215,114.852454,171.059042,224.732834
0.985,172.231000,0.985087,0.014881,1.883828,0.010890,0.910510,4.525510,27.591837,4.229592,2.571429,...,NaN,NaN,NaN,23.384048,561.217146,0.135765,57.630073,114.810207,170.990755,224.630056
0.990,163.652909,0.990268,0.014140,1.881114,0.011482,0.944524,4.857143,25.636905,4.333333,2.392857,...,NaN,NaN,NaN,22.704541,544.908990,0.138674,57.613009,114.774424,170.932938,224.543087


## Add overall summary data to the model file

In [30]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'Power_Turbine1_max']=yearly['PowerTurbine1_max'].max()

models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.at[model_name,'RatedPower']=rated_power_unit

models.loc[model_name]



OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
HeadpondLift                               0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
RatedPower                            49.182
Mean                                  838165
P95                                   719611
P90                                   757122
P75                                   796138
P50                                   843034
P25                                   878919
P10                                   929634
P5        

In [31]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

c:\Users\Andy Fleming\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\stattools.py:657: FutureWarning: The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,0.469047,0.341036,0.282277,0.249693,0.163154,0.140727,0.0886888,0.1689,0.0982952,...,-0.171124,-0.0858771,0.00981251,0.0765236,0.0393113,-0.0223923,0.00191873,0.0127785,-0.0195633,0.0259686
1,"[1.0, 1.0]","[0.26795866633646404, 0.6701347901373955]","[0.0997296178625573, 0.5823425862757071]","[0.02221042830075992, 0.542344190752229]","[-0.02248154971825153, 0.5218667489565847]","[-0.11813042104539992, 0.44443831652208754]","[-0.14435875348458732, 0.4258119812328816]","[-0.19919188607621008, 0.3765694197651849]","[-0.12008335180609236, 0.457883399596651]","[-0.1946527009683468, 0.39124308473530905]",...,"[-0.5361700365400438, 0.19392110562210185]","[-0.45415214384630426, 0.2823979625201615]","[-0.35927141219869635, 0.3788964323765967]","[-0.29257091636554333, 0.4456180257328708]","[-0.3304241926853937, 0.409046720883823]","[-0.3922966902789083, 0.3475121684757115]","[-0.36804050947560973, 0.3718779659878694]","[-0.3571811839557324, 0.38273809628446265]","[-0.38954074108520204, 0.3504142330978421]","[-0.3440506967960879, 0.3959879310403869]"


In [32]:
lowheadshutdowns=daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowheadshutdowns['StartDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['Date']
lowheadshutdowns['EndDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').max()['Date']
lowheadshutdowns['MonthId']=daily[['WaterYear','MonthId']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['MonthId']

#daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').min()#['Date']
lowheadshutdowns['Month_power_max']=monthly.loc[lowheadshutdowns['MonthId']][['WaterYear','Power_max']].set_index('WaterYear')
lowheadshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
WaterYear,,,,,
1933,4,1934-03-25,1934-03-28,1934.03,181.661290
1947,32,1948-03-04,1948-04-04,1948.03,104.685546
1951,20,1952-03-02,1952-03-21,1952.03,103.852197
1952,10,1953-03-21,1953-03-30,1953.03,188.147774
1955,13,1956-05-05,1956-05-17,1956.05,114.089472
1956,27,1957-03-02,1957-03-28,1957.03,104.346731
1957,60,1958-02-03,1958-04-03,1958.02,108.221746
1960,22,1961-04-02,1961-04-23,1961.04,98.033675
1961,11,1962-04-04,1962-04-14,1962.04,102.107043


In [33]:
lowflowshutdowns=daily.loc[daily['LowFlowShutoff']==1].groupby('Year').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowflowshutdowns['StartDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['Date']
lowflowshutdowns['EndDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').max()['Date']
lowflowshutdowns['MonthId']=daily[['Year','MonthId']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['MonthId']


lowflowshutdowns['Month_power_max']=monthly.loc[lowflowshutdowns['MonthId']][['Year','Power_max']].set_index('Year')
lowflowshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
Year,,,,,
1924,10,1924-10-01,1924-10-10,1924.1,18.828159


## Save Files

In [34]:
outp=output_data+out_prefix
#daily.to_csv(outp+'_pe_daily.csv')
calmonthly.to_csv(outp+'_pe_calmonthly.csv')

In [35]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")
    models.to_csv(input_data + "2022/models.csv")
    lowheadshutdowns.to_csv(outp + "lowheadshutdowns.csv")
    lowflowshutdowns.to_csv(outp + "lowflowshutdowns.csv")

In [80]:
years=daily['WaterYear'].unique().tolist()

for year in years:
    days=daily.loc[daily.WaterYear==year]
    #days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 